This notebook illustrates a spell-checking algorithm, that helps restoring abbreviations based on the Lord of the Rings vocabulary!

sampled from: [abbreviation_spellchecker](https://github.com/avidale/weirdMath/blob/master/nlp/abbreviation_spellchecker_Frodo.ipynb)

The problem is how to recover whole words from abbreviations, like these
```
wtrbtl = water bottle
bwlingbl = bowling ball
bsktball = basketball
```
without comprehensive dictionary of whole words.

I use [noisy channel approach](http://web.stanford.edu/~jurafsky/slp3/5.pdf), which considers each abbreviation as a distorted version of the original phrase. 

To recover the original phrase, I need to answer to questions: which distortions are likely, and which phrases are likely.

By Bayes theorem, $p(phrase|abbreviation) \sim p(phrase) p(abbreviation|phrase) = p(phrase) \sum p(distortion|phrase) $, where $distortion$, applied to the original $phrase$, generates the observable phrase - $abbreviation$. 

Both right-hand sided conditional probabilities may be evaluated by NLP models. I will use the simplest class of models - character based n-grams.

With these models, I will perform approximate (beam) search for the most likely originall phrases, letter-by-letter. 

First, I create the language model. 

It merges weighted frequencies of n, n-1, ..., 0 gram-models to give the most stable estimiate of distribution of next each letter in a phrase based on previous letters. It is one of the [vanilla n-gram models](https://en.wikipedia.org/wiki/N-gram).

In [12]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

class LanguageNgramModel:
    """ Remember and predict which letters usually follows which. """
    def __init__(self, order=1, smoothing=1.0, recursive=0.001):
        self.order = order
        self.smoothing = smoothing
        self.recursive = recursive
    
    def fit(self, corpus):
        """ Estimate all counts on a text """
        self.counter_ = defaultdict(lambda: Counter())
        self.unigrams_ = Counter()
        self.vocabulary_ = set()
        for i, token in enumerate(corpus[self.order:]):
            context = corpus[i:(i+self.order)]
            self.counter_[context][token] += 1
            self.unigrams_[token] +=1
            self.vocabulary_.add(token)
        self.vocabulary_ = sorted(list(self.vocabulary_))
        if self.recursive > 0 and self.order > 0:
            self.child_ = LanguageNgramModel(self.order-1, self.smoothing, self.recursive)
            self.child_.fit(corpus)
            
    def get_counts(self, context):
        """ Get smoothed count of each letter appearing after context """
        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        freq_dict = self.counter_[local]
        freq = pd.Series(index=self.vocabulary_)
        for i, token in enumerate(self.vocabulary_):
            freq[token] = freq_dict[token] + self.smoothing
        if self.recursive > 0 and self.order > 0:
            child_freq = self.child_.get_counts(context) * self.recursive
            freq += child_freq
        return freq
    
    def predict_proba(self, context):
        """ Get smoothed probability of each letter appearing after context """
        counts = self.get_counts(context)
        return counts / counts.sum()
    
    def single_log_proba(self, context, continuation):
        """ Estimate log-probability that context is followed by continuation """
        result = 0.0
        for token in continuation:
            result += np.log(self.predict_proba(context)[token])
            context += token
        return result
    
    def single_proba(self, context, continuation):
        """ Estimate probability that context is followed by continuation """
        return np.exp(self.single_log_proba(context, continuation))

Next, I create the missing letter model. It is also based on n-grams, but it estimates probability of each letter being missed based on the previous letters.

This model would be trained on a much smaller dataset, labeled manually. And it will itself be small.

In [13]:
class MissingLetterModel:
    """ Remember and predict which letters are usually missing. """
    def __init__(self, order=0, smoothing_missed=0.3, smoothing_total=1.0):
        self.order = order
        self.smoothing_missed = smoothing_missed
        self.smoothing_total = smoothing_total
    def fit(self, sentence_pairs):
        self.missed_counter_ = defaultdict(lambda: Counter())
        self.total_counter_ = defaultdict(lambda: Counter())
        for (original, observed) in sentence_pairs:
            for i, (original_letter, observed_letter) in enumerate(zip(original[self.order:], observed[self.order:])):
                context = original[i:(i+self.order)]
                if observed_letter == '-':
                    self.missed_counter_[context][original_letter] += 1
                self.total_counter_[context][original_letter] += 1 
    def predict_proba(self, context, last_letter):
        """ Estimate probability that last_letter after context is missed """
        if self.order:
            local = context[-self.order:]
        else:
            local = ''
        missed_freq = self.missed_counter_[local][last_letter] + self.smoothing_missed
        total_freq = self.total_counter_[local][last_letter] + self.smoothing_total
        return missed_freq / total_freq
    
    def single_log_proba(self, context, continuation, actual=None):
        """ Estimate log-probability of continuaton being distorted to actual after context. 
        If actual is None, assume no distortion
        """
        if not actual:
            actual = continuation
        result = 0.0
        for orig_token, act_token in zip(continuation, actual):
            pp = self.predict_proba(context, orig_token)
            if act_token == '-':
                pp = 1 - pp
            result += np.log(pp)
            context += orig_token
        return result
    
    def single_proba(self, context, continuation, actual=None):
        """ Estimate probability of continuaton being distorted to actual after context. 
        If actual is None, assume no distortion
        """
        return np.exp(self.single_log_proba(context, continuation, actual))

Let's start wiht a simple example. I train my language model on a single word.

In [14]:
lang_model = LanguageNgramModel(1)
lang_model.fit(' abracadabra ')
lang_model.predict_proba(' bra')

     0.181777
a    0.091297
b    0.272529
c    0.181686
d    0.181686
r    0.091025
dtype: float64

Next, I train my distortion model on a single (original word, distortion) pair.

In [15]:
missed_model = MissingLetterModel(0)
missed_model.fit([('abracadabra', 'abr-c-d-br-')]) 
missed_model.predict_proba('abr', 'a'), missed_model.predict_proba('abr', 'b')

(0.7166666666666667, 0.09999999999999999)

In [16]:
missed_model.single_proba('', 'abra', 'abr-')

0.0020305555555555532

**Approach to the search**

For simplicity, we let each query (distorted phrase) start and end with whitespaces (beginning and end of phrases).

We will have an infinite tree which we hant to explore for the best (most probable) path from the root to a leaf.

The root is the beginning of the phease. Each edge is an additional letter, and it can be missed or intact. Each edge is assigned probability conditional on several previous edges. Thus probability of each path is the product of probabilities of its edges (or sum of log-probabilities).

A node is declared as a leaf, if its path, with the missed letters dropped, equals the query. 

**Search algorithm**

This tree is possibly infinite, but we need only the good leaves. So we do a kind of beam search: 
 * at each node, estimate log-probability of its ancestor leaves as $optimism \times default$, where $optimism$ is a user-provided coefficient, and $default$ is the unconditional probability of the corresponding (unprocessed) phrase suffix at this node. 
 * look only at the nodes with estimate of log-likelihood no less than best current log-likelihood minus $freedom$.
 
So, the lower $optimism$ and the higher $freedom$, the slower the search will be, and the more paths will be explored.

In [17]:
from heapq import heappush, heappop

This function generates children for each node and estimates likelihood of their ancestor leaves.

In [18]:
def generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism=0.5, cache=None):
    options = []
    for letter in lang_model.vocabulary_ + ['']:
        if letter:  # assume a missing letter
            next_letter = letter
            new_suffix = suffix
            new_prefix = prefix + next_letter
            proba_missing_state = - np.log(missed_model.predict_proba(prefix, letter))
        else:  # assume no missing letter
            next_letter = suffix[0]
            new_suffix = suffix[1:]
            new_prefix = prefix + next_letter
            proba_missing_state = - np.log((1 - missed_model.predict_proba(prefix, next_letter)))
        proba_next_letter = - np.log(lang_model.single_proba(prefix, next_letter))
        if cache:
            proba_suffix = cache[len(new_suffix)] * optimism
        else:
            proba_suffix = - np.log(lang_model.single_proba(new_prefix, new_suffix)) * optimism
        proba = prefix_proba + proba_next_letter + proba_missing_state + proba_suffix
        options.append((proba, new_prefix, new_suffix, letter, proba_suffix))
    return options
print(generate_options(0, ' ', 'brac ', lang_model, missed_model))

[(6.929663174828117, '  ', 'brac ', ' ', 3.7800651217336947), (5.042879645338754, ' a', 'brac ', 'a', 3.4572571306016755), (8.09487194753453, ' b', 'brac ', 'b', 3.846661605771999), (7.623807861705187, ' c', 'brac ', 'c', 3.7800651217336947), (7.623807861705187, ' d', 'brac ', 'd', 3.7800651217336947), (8.09487194753453, ' r', 'brac ', 'r', 3.846661605771999), (4.858238261775765, ' b', 'rac ', '', 2.8072524973494524)]


This function explores the graph on noisy channel in the best-first manner, until it runs out of attempts or out of optimistic nodes.

In [19]:
def noisy_channel(word, lang_model, missed_model, freedom=1.0, max_attempts=1000, optimism=0.1, verbose=True):
    query = word + ' '
    prefix = ' '
    prefix_proba = 0.0
    suffix = query
    full_origin_logprob = -lang_model.single_log_proba(prefix, query)
    no_missing_logprob = -missed_model.single_log_proba(prefix, query)
    best_logprob = full_origin_logprob + no_missing_logprob
    # add empty beginning to the heap
    heap = [(best_logprob * optimism, prefix, suffix, '', best_logprob * optimism)]
    # add the default option (no missing letters) to candidates
    candidates = [(best_logprob, prefix + query, '', None, 0.0)]
    if verbose:
        # todo: include distortion probability
        print('baseline score is', best_logprob)
    # prepare cache for suffixes (the slowest operation)
    cache = {}
    for i in range(len(query)+1):
        future_suffix = query[:i]
        cache[len(future_suffix)] = -lang_model.single_log_proba('', future_suffix) # rough approximation
        cache[len(future_suffix)] += -missed_model.single_log_proba('', future_suffix) # at least add missingness
    
    for i in range(max_attempts):
        if not heap:
            break
        next_best = heappop(heap)
        if verbose:
            print(next_best)
        if next_best[2] == '':  # it is a leaf
            # this is the best leaf as far, add it to candidates
            if next_best[0] <= best_logprob + freedom:
                candidates.append(next_best)
                # update the best likelihood
                if next_best[0] < best_logprob:
                    best_logprob = next_best[0]
        else: # it is not a leaf - generate more options
            prefix_proba = next_best[0] - next_best[4] # all proba estimate minus suffix
            prefix = next_best[1]
            suffix = next_best[2]
            new_options = generate_options(prefix_proba, prefix, suffix, lang_model, missed_model, optimism, cache)
            # add only the solution potentioally no worse than the best + freedom
            for new_option in new_options: 
                if new_option[0] < best_logprob + freedom:
                    heappush(heap, new_option)
    if verbose:
        print('heap size is', len(heap), 'after', i, 'iterations')
    result = {}
    for candidate in candidates:
        if candidate[0] <= best_logprob + freedom:
            result[candidate[1][1:-1]] = candidate[0]
    return result

We apply this function to the abbreviation 'brc' and look for suggested options with scores (the lower the better)

In [20]:
result = noisy_channel('brc', lang_model, missed_model, freedom=2.0, optimism=0.5, verbose=True)
print(result)

baseline score is 14.659531132722798
(7.329765566361399, ' ', 'brc ', '', 7.329765566361399)
(7.729102491649175, ' b', 'rc ', '', 5.6781167272228625)
(6.82819709010665, ' br', 'c ', '', 3.689648873198813)
(7.4281382278577714, ' brc', ' ', '', 2.0472553582899407)
(7.68318306227505, ' brc ', '', '', -0.0)
(8.142544971129297, ' bra', 'c ', 'a', 3.689648873198813)
(8.36814476033081, ' brac', ' ', '', 2.0472553582899407)
(8.623189594748087, ' brac ', '', '', -0.0)
(8.838538268507152, ' a', 'brc ', 'a', 7.252915753770074)
(8.669109024122214, ' ab', 'rc ', '', 5.6781167272228625)
(7.768203622579689, ' abr', 'c ', '', 3.689648873198813)
(8.36814476033081, ' abrc', ' ', '', 2.0472553582899407)
(8.623189594748087, ' abrc ', '', '', -0.0)
(9.013760742594851, ' brca', ' ', 'a', 2.0472553582899407)
(9.028155327065601, ' brca ', '', '', -0.0)
(9.082551503602335, ' abra', 'c ', 'a', 3.689648873198813)
(9.30815129280385, ' abrac', ' ', '', 2.0472553582899407)
(9.563196127221126, ' abrac ', '', '', -0.

# Train model on corpus 

Now let's train a good language model on a large corpus - Lord of the Rings!

To start, drop all characters except spaces and letters.

In [21]:
with open('corpus\Corpus.txt', encoding = 'utf-8') as f:
    text = f.read()
import re
text2 = re.sub(r'[^a-z ]+', '', text.lower().replace('\n', ' '))
print(text2[0:100])

annual report  petronas chemicals group berhad wwwpetronaschemicalscom shareholder value environment


In [22]:
all_letters = ''.join(list(sorted(list(set(text2)))))
print(repr(all_letters))

' abcdefghijklmnopqrstuvwxyz'


Prepare a small training corpus for the missing-word model, that shows that letters 'aeiouy' are missed more frequently 

In [23]:
missing_set = [
] + [(all_letters, '-' * len(all_letters))] * 3 + [(all_letters, all_letters)] * 10 + [('aeiouy', '------')] * 30

Choose the best model order by comparing log likelihoods on the end of the book ('test set'). 

The longer memory, the better. But after order 4 the gain is not so large, so we stop here.

In [24]:
for i in range(5):
    tmp = LanguageNgramModel(i, 1.0, 0.001)
    tmp.fit(text2[0:-5000])
    print(i, tmp.single_log_proba(' ', text2[-5000:]))

0 -14306.373304419854
1 -12687.401358449732
2 -10331.869983054516
3 -7416.3294112879385
4 -6018.242023578867


Train the 5-gram language model and 1-gram missing letter model.

In [25]:
big_lang_m = LanguageNgramModel(4, 0.001, 0.01)
big_lang_m.fit(text2)
big_err_m = MissingLetterModel(0, 0.1)
big_err_m.fit(missing_set)

Apply our algorithm to different abbreviations:

In [29]:
noisy_channel('ptro', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'petrol': 11.94817530941005, 'petronas': 10.679361014950668}

In [27]:
noisy_channel('enty key', big_lang_m, big_err_m, max_attempts=10000, optimism=0.9, freedom=3.0, verbose=False)

{'enty key': 32.12997944715083}

In [41]:
noisy_channel('enty key', big_lang_m, big_err_m, max_attempts=10000, optimism=0.5, freedom=0.5, verbose=False)

{'entify key': 25.13389562698879}

In [45]:
noisy_channel('enty key', big_lang_m, big_err_m, max_attempts=10000, optimism=0.7, freedom=0.1, verbose=False)

{'entry key': 28.68073178382995}

However, with the relevant training corpus the sports words would be recognized.